I want this to be the single best and most complete document that exists on how to make money using triangular arbitrage.

## Roadmap

1. Absolute Basics - What is Triangular Arbitrage?
2. Algorithmic Solution
3. Implementation Details
4. Real world tricks

## Basics

Let's break down the term triangular arbitrage. First, what's arbitrage? Basically it's a fancy word for buying low and selling high.

Pretend you are walking down the street and you see an open farmer's market where people are buying and selling bread for \\$1 per loaf. You keep walking and 5 minutes later you see another market this time selling bread for \\$2 per loaf. Displaying unrivalled business accumen, you decide to head back and buy as many \\$1 loaves as your greedy little hands can carry. You bring them to the new market, sell them for \\$2 and make a clean buck for each loaf.

You just completed bread arbitrage.

And made every finance researcher angry. Because many financial theories assume that this situation won't happen. They assume that if there is a price difference in equivalent assets (same bread), then smart individuals like yourselves will buy cheap and sell at the expensive place, until the prices equal out. Imagine if whole hordes of people went to the \\$1 place, trying to buy as many loaves as possible. The sellers would quickly raise their prices.

And equilibrium is reached when both markets have the same price, \\$1.50 per loaf.

Let's make this example a little more relevent to our future topic.

Pretend that you own 1 USD. And you can buy 1.5 CAD with 1 USD on Bob's exchange. Jim's exchange will let you sell 1.5 CAD for 1.1 USD. Now you buy 1.5 CAD on Bob's exchange and immediately sell it on Jim's exchange. You end up with 1.1 USD.

You made 10 cents for free! This is currency arbitrage.

#### Triangular?

Now we get arbitrage. Where does the triangular part come in?

Well, it means doing our currency arbitrage like above, but using 3 different currencies instead of just USD and CAD. Here's an example.

Exchange Name | Trade Avialable
--- | ---
Bob's | 1 USD for 1.5 CAD
Jim's | 1.5 CAD for 2 Euros
Ted's | 2 Euros for 1.1 USD

Now we buy CAD at Bob's exchange, trade that for Euro's at Jim's and go back to USD at Ted's, making a sweet 10 cents.

This seems crazy simple. And it is... when I've layed it out so nicely for you. Things get more complicated when looking at real exchange data because you have to keep in mind exchange ratios.

As a reminder, if you have 1 USD to 1.5 CAD, you can find it in terms of 1 CAD by dividing by 1.5. so you get 0.67 USD for 1 CAD. It's helpful to think of it as a ratio—1.5 CAD to 1 USD. Then divide both sides by 1.5 to get 1 CAD to 0.67 USD.

Now let's work through the full example.

_        | USD  | CAD  | Euro
---      | ---  | ---  | ---
**USD**  | 1    | 1.5  | 1.9
**CAD**  | 0.67 | 1    | 1.33
**Euro** | 0.55 | 0.75 | 1

This table means we can trade 1 USD for 1.5 CAD or 1.9 Euro. And we can trade 1 Euro for 0.55 USD and 0.75 CAD.

In this example, starting with USD we go 1 USD for 1.5 CAD. That 1.5 CAD is traded for 1.5 $\times$ 1.33 = 1.995 Euros. The 1.995 Euros are traded for 1.995 $\times$ 0.55 = 1.10 USD.

But how did I know that USD to CAD to Euros to USD was going to make money? Why not USD to Euro to CAD to USD? Does order matter? Well the simplified math of USD to Euro to CAD to USD is 1 $\times$ 1.9 $\times$ 0.75 $\times$ 0.67 = 0.95 USD. You lost 5 cents on the round trip.

Now, typical exchanges have tons of currencies. Instead of a 3x3 table, you have a 100x100 table. How do you know which series of trades is going to work, if any? What if there is some crazy series of 10 trades in a cycle?

## Algorithm

This is an excellent problem for a computer. This problem can be formulated as a graph search problem. If you don't know anything about computer science or algorithms, just skip to the next section.

We can visuallize the problem as a weighted graph. Each currency corresponds to a node, while the exchange rates correspond to edges. The weight of an edge from node $u$ to $v$ is the amount of $v$ we can buy with 1 $u$. Then an arbitrage oppurtunity exists if we find a cycle in the graph where the edges multiply to more than 1.

Now we can use graph search algorithms to find this cycle. However there is a problem. The shortest path algorithms assume the edges are summed together. Triangular arbitrage is multiplicative. We calculate the total amount of USD we have at the end by multiplying all the exchange rates together.

Thus, in order to turn multiplication into addition, we set the edges to be the log of the exchange rate.

The other problem is that really we are trying to find the **longest** path. Because we assume that most series of trades will not be profitable and will result in a final trip value of less than 1. We want the series of trades that results in the largest value. Thus we set edges to be the negative log of exchange rates. Now shortest path algorithms will find the smallest value, which will be our most profitable trade.

As we are finding the shortest path in a graph with negative edge weights, the bellman-ford algorithm is the best choice. It finds the shortest path to all nodes from a starting node and can detect cycles where the edges sum to a negative value. 

The math for triangular arbitrage is like so:

When $R_{ij}$ is the exchange rate between currency $i$ and $j$ then we have arbitrage if

$$R_{1,2} \times R_{2,3} \times R_{3,4} \times \ldots \times R_{k-1,k} \times R_{k,1} > 1$$

$$\log R_{1,2} + \log R_{2,3} + \log R_{3,4} + \ldots + \log R_{k-1,k} + \log R_{k,1} > 0$$

$$(-\log R_{1,2}) + (-\log R_{2,3}) + (-\log R_{3,4}) + \ldots + (-\log R_{k-1,k}) + (-\log R_{k,1}) < 0$$

### Bellman Ford

Bellman Ford sets the distance to the starting node to 0. Then all other nodes get a distance of infinity. Then it looks at all edges and checks if the distance to the node connected to the edge is shorter if it takes the edge. Then it updates the distance. Now it has all the shortest paths of length 1. And at each iteration i, it scans all edges and finds all the new shortest paths of at most length i. It does this for |V| - 1 times as that is the longest possible path with no cycle. Then it checks one more time. If any distances are updated then it knows that the path is length |V|, which could only occur if there is a negative cycle.

## Code

Let's see this in code:

In [23]:
G = {
    'USD': {'CAD': 1.5, 'EUR': 1.9},
    'CAD': {'USD': 0.67, 'EUR': 1.33},
    'EUR': {'USD': 0.55, 'CAD': 0.75}
    }

def bellman_ford(G, source):
    distance = dict.fromkeys(G, float('inf'))
    distance[source] = 0
 
    for _ in range(len(G) - 1):
        for currency in G:
            for neighbour in G[currency]:
                distance[neighbour] = min(distance[neighbour], distance[currency] + G[currency][neighbour])
 
    return distance

In [24]:
bellman_ford(G, 'USD')

{'USD': 0, 'CAD': 1.5, 'EUR': 1.9}

In [ ]:
G = Graph()
G.nodes['USD'] = Node('USD')
G.nodes['CAD'] = Node('CAD')
G.nodes['EUR'] = Node('EUR')
G.add_edge('USD', 'CAD', 1.5)
G.add_edge('USD', 'EUR', 1.9)
G.add_edge()

In [5]:
import math
def is_arbitrage_exists(G):
    def bellman_ford(G, source):
        dis_to_source = ([float('inf')] * (source - 1) + [0] + [float('inf')] * (len(G) - source))
    
        for _ in range(1, len(G)):
            have_update = False
            for i, row in enumerate(G):
                for j, g in enumerate(row):
                    if (dis_to_source[i] != float('inf') and dis_to_source[j] > dis_to_source[i] + g):
                        have_update = True
                        dis_to_source[j] = dis_to_source[i] + g
            if not have_update:
                return False
            return any(dis_to_source[i] != float('inf')
                      and dis_to_source[j] > dis_to_source[i] + g
                      for i, row in enumerate(G) for j, g in enumerate(row))
    return bellman_ford([[-math.log(edge) for edge in edge_list] for edge_list in G], 0)

In [16]:
G = [[1, 1.5, 1.9],
    [0.67, 1, 1.33],
    [0.55, 0.75, 1]]
is_arbitrage_exists(G)

True


Now one problem you face in our bread situation is you might buy a bunch of bread for \\$1, only to arrive at the \\$2 market and find the price has dropped to the equilibrium price. Or no one is buying. That would suck.

Which is why usually arbitrage refers to the simultaneous buying and selling, to guarantee the profit.
